In [1]:
import os
from glob import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import cv2

/misc/me/pratikm/virtualEnv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Global variables

In [65]:
DIR_TFRECORDS = 'data_small_tfrecords'
DATA_TRAIN = glob('./'+DIR_TFRECORDS+'/[0-9].tfrecords')

NUM_OBJECTS = 20
MAX_DETECTIONS_PER_IMAGE = 20

GRID_SIZE = 608#32
GRID_H, GRID_W = 1,1#19, 19

ANCHORS_NORMALIZED = np.array(
    [
        [0.09112895, 0.06958421],
        [0.21102316, 0.16803947],
        [0.42625895, 0.26609842],
        [0.25476474, 0.49848   ],
        [0.52668947, 0.59138947]
    ]
)
ANCHORS = ANCHORS_NORMALIZED * np.array([GRID_H, GRID_W])
NUM_ANCHORS = ANCHORS.shape[0]

IMG_H, IMG_W = 608, 608  # GRID_H * GRID_SIZE = 19 * 32 = 608

COOEFFICIENT_OBJ = 1
COOEFFICIENT_NO_OBJ = 1
COOEFFICIENT_REG = 5

THRESHOLD_IOU_SCORES = 0.5
COEFF_LOSS_CONFIDENCE_OBJECT_PRESENT = 5
COEFF_LOSS_CONFIDENCE_OBJECT_ABSENT = 1
THRESHOLD_OUT_PROB = 0.5
THRESHOLD_IOU_NMS = 0.5

NUM_EPOCHS = 10
BATCH_SIZE = 4
MODEL_DIR = 'model'

DEBUG = False

## Yolo Loss function

In [52]:
def apply_transformations(predictions):
    predictions_yx = tf.sigmoid(predictions[..., 0:2])
    predictions_hw = tf.exp(predictions[...,2:4])
    predictions_prob_obj = tf.sigmoid(predictions[...,4:5])
    predictions_prob_class = tf.nn.softmax(predictions[...,5:])
    
    return predictions_yx, predictions_hw, predictions_prob_obj, predictions_prob_class

def get_coordinates(h, w):
    coordinates_y = tf.range(h)
    coordinates_x = tf.range(w)
    x, y = tf.meshgrid(coordinates_x, coordinates_y)
    coordinates = tf.stack([y,x], axis=-1)
    coordinates = tf.reshape(coordinates, [1, h, w, 1, 2])
    coordinates = tf.cast(coordinates, tf.float32)
    
    return coordinates

def grid2normalized(predictions_yx, predictions_hw):    
    # create cartesian coordinates on grid space
    coordinates = get_coordinates(GRID_H, GRID_W)
    
    # map from grid space to [0,19] space
    anchors = tf.cast(tf.reshape(ANCHORS, [1, 1, 1, ANCHORS.shape[0], 2]), dtype=tf.float32)  # [0,19] space
    predictions_yx += coordinates
    predictions_hw *= anchors
    
    # map from [0,19] space to [0,1] space
    shape = tf.cast(tf.reshape([GRID_H, GRID_W], [1, 1, 1, 1, 2]), tf.float32)
    predictions_yx /= shape
    predictions_hw /= shape
    
    return predictions_yx, predictions_hw

def get_boxes_gt(args_map):
    # extract ground truth bboxes wherever prob_obj = 1
    mask_object = tf.cast(tf.reshape(args_map[1], [GRID_H, GRID_W, NUM_ANCHORS]), tf.bool)
    bboxes = tf.boolean_mask(args_map[0], mask_object)
    # bboxes.shape = [NUM_DETECTIONS, 4]; NUM_DETECTIONS vary with each image
    
    # pad bboxes so that bboxes is fixed dimension (fix NUM_DETECTIONS to MAX_DETECTIONS_PER_IMAGE)
    pad = tf.zeros((MAX_DETECTIONS_PER_IMAGE - tf.shape(bboxes)[0], 4))  # TODO: when NUM_DETECTIONS > MAX_DETECTIONS_PER_IMAGE
    bboxes = tf.concat([bboxes, pad], axis=0)
    
    return bboxes

def get_iou_scores(predictions_yx, predictions_hw, bboxes_gt):
    # predictions_yx.shape = predictions_hw.shape = [BATCH_SIZE, GRID_H, GRID_W, NUM_ANCHORS, 2]
    # bboxes_gt.shape = [BATCH_SIZE, MAX_DETECTIONS_PER_IMAGE, 4]
    
    # compute ious for each anchor in each grid in axis=4
    predictions_yx = tf.expand_dims(predictions_yx, 4)
    predictions_hw = tf.expand_dims(predictions_hw, 4)
    
    predictions_min = predictions_yx - predictions_hw/2.
    predictions_max = predictions_yx + predictions_hw/2.
    
    bboxes_gt = tf.reshape(bboxes_gt, [tf.shape(bboxes_gt)[0], 1, 1, 1, MAX_DETECTIONS_PER_IMAGE, 4])
    bboxes_gt_yx = bboxes_gt[..., 0:2]
    bboxes_gt_hw = bboxes_gt[..., 2:4]
    
    bboxes_gt_min = bboxes_gt_yx - bboxes_gt_hw/2.
    bboxes_gt_max = bboxes_gt_yx + bboxes_gt_hw/2.
    
    intersection_min = tf.maximum(predictions_min, bboxes_gt_min)
    intersection_max = tf.minimum(predictions_max, bboxes_gt_max)
    intersection_hw = tf.maximum(intersection_max - intersection_min, 0.)
    area_intersection = intersection_hw[..., 0] * intersection_hw[..., 1]
    
    area_predictions = predictions_hw[...,0] * predictions_hw[...,1]
    area_bboxes_gt = bboxes_gt_hw[...,0] * bboxes_gt_hw[...,1]
    area_union = area_bboxes_gt + area_predictions - area_intersection
    iou = area_intersection / area_union
    
    return iou

def get_confidence_loss(labels_prob_obj, iou_mask, predictions_prob_obj):
    mask_object_absent = (1 - labels_prob_obj) * (1 - iou_mask)
    loss_object_absent = mask_object_absent * tf.square(predictions_prob_obj)
    
    loss_object_present = labels_prob_obj * tf.square(1-predictions_prob_obj)
    
    loss_confidence = COEFF_LOSS_CONFIDENCE_OBJECT_ABSENT * loss_object_absent \
            + COEFF_LOSS_CONFIDENCE_OBJECT_PRESENT * loss_object_present
    
    return tf.reduce_sum(loss_confidence)
    
def get_classification_loss(labels_prob_obj, labels_class, predictions_prob_class):
    labels_class = tf.cast(labels_class, tf.int32)
    labels_class = tf.one_hot(labels_class, NUM_OBJECTS)
    
    loss_classification = labels_prob_obj * tf.squared_difference(labels_class, predictions_prob_class)
    
    return tf.reduce_sum(loss_classification)

def get_regression_loss(labels_bbox, predictions_bbox, labels_prob_obj):
    loss_regression = labels_prob_obj * tf.squared_difference(labels_bbox,predictions_bbox)
    
    return tf.reduce_sum(loss_regression)

def get_loss(predictions, labels): 
    # predictions.shape = [BATCH_SIZE, GRID_H, GRID_W, NUM_ANCHORS, 5+NUM_OBJECTS] (they are in grid space)
    # labels.shape = [BATCH_SIZE, GRID_H, GRID_W, NUM_ANCHORS, 6]
    
    # apply corresponding transformations on predictions
    predictions_yx, predictions_hw, predictions_prob_obj, predictions_prob_class = apply_transformations(predictions)
    
    # map predictions_bbox to [0,1] space
    predictions_yx, predictions_hw = grid2normalized(predictions_yx, predictions_hw)
    
    # map labels_bbox to [0,1] space
    labels_yx, labels_hw = grid2normalized(labels[...,0:2], labels[...,2:4])
    
    # get ground truth bboxes using labels_bbox & prob_obj in labels
    labels_bbox = tf.concat([labels_yx, labels_hw], axis=-1)
    bboxes_gt = tf.map_fn(get_boxes_gt, (labels_bbox, labels[...,5]), dtype=tf.float32)
    
    # compute iou scores for each anchor in each grid for all bboxes_gt
    iou_scores = get_iou_scores(predictions_yx, predictions_hw, bboxes_gt)
     
    # keep anchors whose iou_scores are above THRESHOLD_IOU_SCORES
    iou_scores_best = tf.reduce_max(iou_scores, axis=4, keep_dims=True)
    iou_mask = tf.cast(iou_scores_best > THRESHOLD_IOU_SCORES, tf.float32)
    
    
    ## Loss
    # object confidence loss (presence and absence)
    loss_confidence = get_confidence_loss(labels[...,5:6], iou_mask, predictions_prob_obj)
    
    # classification loss
    loss_classification = get_classification_loss(labels[...,5:6], labels[...,4], predictions_prob_class)
    
    # regression loss
    predictions_bbox = tf.concat([predictions_yx, predictions_hw], axis=-1)
    loss_regression = get_regression_loss(labels_bbox, predictions_bbox, labels[...,5:6])
    
    # total loss
    loss = ( loss_confidence + loss_classification + loss_regression ) / tf.cast(tf.shape(labels)[0], tf.float32)
    
    return loss

## Estimator

In [67]:
def parse_record(record):
    # dictionary as per saved TFRecord
    keys_to_features = {
        'img': tf.FixedLenFeature(shape=(), dtype=tf.string),
        'label': tf.FixedLenFeature(shape=(), dtype=tf.string),
    }

    # parse record
    parsed = tf.parse_single_example(record, keys_to_features)

    # decode image
    img = tf.decode_raw(parsed['img'], tf.uint8)
    img = tf.cast(tf.reshape(img, [IMG_H, IMG_W, 3]), tf.float32)
    img /= 255.  # normalize

    # decode label
    label = tf.decode_raw(parsed['label'], tf.float32)
    label = tf.reshape(label, [GRID_H, GRID_W, NUM_ANCHORS, 6])

    return {'img': img}, label

def input_fn_train(data_file):    
    # dataset processing
    dataset = tf.data.TFRecordDataset(data_file)
    
    dataset = dataset.map(parse_record)
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat()
    
    iterator = dataset.make_one_shot_iterator()
    img, label = iterator.get_next()
    
    return img, label

def input_fn_predict(data_file):    
    # dataset processing
    dataset = tf.data.TFRecordDataset(data_file)
    
    dataset = dataset.map(parse_record)
    dataset = dataset.batch(BATCH_SIZE)
    
    iterator = dataset.make_one_shot_iterator()
    img, label = iterator.get_next()
    
    return img, label

def get_feature_columns():
    img = tf.feature_column.numeric_column('img', dtype=tf.float32)
    
    return img

def model_fn(features, labels, mode, params):
    # input layer
    features = tf.feature_column.input_layer(features, params['feature_columns'])
    imgs = tf.reshape(features, [-1, IMG_H, IMG_W, 3])
    # imgs.shape = (BATCH_SIZE, 608, 608, 3)
    
    # dense
    x = tf.image.resize_images(imgs, [64, 64])
    x = tf.layers.flatten(x)
    x = tf.layers.dense(x, 256, activation=tf.nn.relu, kernel_initializer=tf.glorot_uniform_initializer())
    x = tf.layers.dense(x, 256, activation=tf.nn.relu, kernel_initializer=tf.glorot_uniform_initializer())
    x = tf.layers.dense(x, GRID_H*GRID_W*NUM_ANCHORS*(5+NUM_OBJECTS))
    predictions = tf.reshape(x, [-1, GRID_H, GRID_W, NUM_ANCHORS, 5+NUM_OBJECTS])
    
#     # CNN
#     with tf.name_scope('CNN'):
#         x = tf.layers.conv2d(imgs, filters=8, kernel_size=5, activation=tf.nn.relu, padding='same')
#         x = tf.layers.max_pooling2d(x, pool_size=4, strides=4)
        
#         # x.shape = (BATCH_SIZE, 152, 152, 8)

#         x = tf.layers.conv2d(x, filters=64, kernel_size=3, activation=tf.nn.relu, padding='same')
#         x = tf.layers.max_pooling2d(x, pool_size=4, strides=4)
#         # x.shape = (BATCH_SIZE, 38, 38, 64)

#         x = tf.layers.conv2d(x, filters=125, kernel_size=3, activation=tf.nn.relu, padding='same')
#         x = tf.layers.max_pooling2d(x, pool_size=2, strides=2)
#         # x.shape = (BATCH_SIZE, 19, 19, 125)

#         predictions = tf.reshape(x, (-1, GRID_H, GRID_W, params['num_anchors'], 5+params['num_objects']))
#         # output.shape = (BATCH_SIZE, 19, 19, 5, 25)
    
    ## PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        
        outputs = {
            'imgs': imgs,
            'predictions': predictions
        }
        return tf.estimator.EstimatorSpec(mode, predictions=outputs)
    
    # compute loss
    with tf.name_scope('Loss'):
        loss = get_loss(predictions, labels)
    tf.summary.scalar('loss_summary', loss)
    
    ## EVALUATE
#     # compute evaluation metrics
#     mse = tf.metrics.mean_squared_error(labels=labels, predictions=predictions, name='mse_op')
#     metrics = {'mse': mse}
#     tf.summary.scalar('mse', mse[1])
#     if mode == tf.estimator.ModeKeys.EVAL:
#         return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
    
    ## TRAIN
    assert mode == tf.estimator.ModeKeys.TRAIN
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(0.0001)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

## Model

In [68]:
# get feature columns
feature_columns = get_feature_columns()

# training configuration
run_config = tf.estimator.RunConfig().replace(keep_checkpoint_max=1, save_summary_steps=10)

# define model
model = tf.estimator.Estimator(
    model_fn=model_fn,
    params={
        'feature_columns': feature_columns,
        'num_anchors': ANCHORS.shape[0],
        'num_objects': NUM_OBJECTS
    },
    model_dir=MODEL_DIR,
    config=run_config
)

INFO:tensorflow:Using config: {'_session_config': None, '_task_id': 0, '_keep_checkpoint_max': 1, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_global_id_in_cluster': 0, '_save_summary_steps': 10, '_model_dir': 'model', '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d4f01f2b0>, '_service': None, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_master': ''}


## Train

In [70]:
model.train(input_fn=lambda: input_fn_train(DATA_TRAIN), steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into model/model.ckpt.
INFO:tensorflow:step = 1001, loss = 0.0021196129
INFO:tensorflow:global_step/sec: 64.6897
INFO:tensorflow:step = 1101, loss = 0.0013777023 (1.546 sec)
INFO:tensorflow:global_step/sec: 73.0274
INFO:tensorflow:step = 1201, loss = 0.021707235 (1.369 sec)
INFO:tensorflow:global_step/sec: 71.8155
INFO:tensorflow:step = 1301, loss = 0.37911776 (1.392 sec)
INFO:tensorflow:global_step/sec: 72.9915
INFO:tensorflow:step = 1401, loss = 0.12756398 (1.370 sec)
INFO:tensorflow:global_step/sec: 73.0025
INFO:tensorflow:step = 1501, loss = 0.0681815 (1.370 sec)
INFO:tensorflow:global_step/sec: 72.2343
INFO:tensorflow:step = 1601, loss = 0.00

## Predict

In [ ]:
predictions = model.predict(input_fn=lambda: input_fn_predict(DATA_TRAIN))

idx = 0
iteration = 0
for p in predictions:
    img, p = p['imgs'], p['predictions']
    if iteration == idx:
        break
    iteration += 1

img = draw_adjusted_anchors(img, p, 'pred')
plt.imshow(img)
plt.show()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(p[0,0,:,0:5])

## Utility

In [16]:
def sigmoid(x):
    return np.exp(x)/(1+np.exp(x))

def draw_adjusted_anchor(img, idx_h, idx_w, idx_a, label_bbox):
    # box_adjustment.shape = [5,]
    
    center_y = (label_bbox[0]+idx_h)*GRID_SIZE
    center_x = (label_bbox[1]+idx_w)*GRID_SIZE
    
    height = label_bbox[2] * ANCHORS[idx_a,0] * GRID_SIZE
    width = label_bbox[3] * ANCHORS[idx_a,1] * GRID_SIZE
     
    left = int(center_x - width/2)
    top = int(center_y - height/2)
    right = int(left + width)
    bottom = int(top + height)

    img = cv2.rectangle(img, (left, top), (right, bottom), color=(255, 0, 0), thickness=3)
    
    return img
    
def draw_adjusted_anchors(img, label, data='gt'):
    # unnormalize image
    img = (img * 255).astype(np.uint8)
    
    for idx_h in range(GRID_H):
        for idx_w in range(GRID_W):
            for idx_a in range(NUM_ANCHORS):
                if sigmoid(label[idx_h, idx_w, idx_a, 5]) > 0:
                    bbox = label[idx_h, idx_w, idx_a, :4]
                    if data == 'pred':
                        bbox[0:2] = sigmoid(bbox[0:2])  # to keep offsets in interval [0,1]
                        bbox[2:4] = np.exp(bbox[2:4])  # to keep width and height to be positive
                    
                    img = draw_adjusted_anchor(img, idx_h, idx_w, idx_a, bbox)
    
    return img

In [ ]:
def prediction2output(prediction):
    # get boxes (bbox, class)
    boxes = get_boxes(prediction)
    
    # perform nms
    boxes_nms = perform_nms(boxes)
    
    # draw boxes on image
    

## Post-processing

In [ ]:
def center2corner(out_yx, out_hw):
    # out_yx.shape = [GRID_H, GRID_W, NUM_ANCHORS, 2]
    
    box_min = out_yx - (out_hw/2.)
    box_max = out_yx + (out_hw/2.)
    
    out_corners = tf.concat([box_min[...,0:1], box_min[...,1:2], box_max[...,0:1], box_max[...,1:2]], axis=-1)
    return out_corners

def get_filtered_output(map_params):
    # this is done to use batch 
    out_corners, out_prob_obj, out_prob_class = map_params[0], map_params[1], map_params[2]
    
    # compute overall prob for each anchor in each grid
    out_prob = out_prob_obj * out_prob_class
    
    # get max prob among all classes at each anchor in each grid
    class_max = tf.argmax(out_prob, axis=-1)
    prob_class = tf.reduce_max(out_prob, axis=-1)
    
    # compute filter mask
    mask_filter = prob_class >= THRESHOLD_OUT_PROB
    
    # apply mask on output
    box_filtered = tf.boolean_mask(out_corners, mask_filter)
    prob_filtered = tf.boolean_mask(prob_class, mask_filter)
    class_filtered = tf.boolean_mask(class_max, mask_filter)
    return box_filtered, prob_filtered, class_filtered

def get_predictions(output):
    # output.shape = [BATCH_SIZE, GRID_H, GRID_W, NUM_ANCHORS, 5+20]
    
    # split output
    out_yx, out_hw, out_prob_obj, out_prob_class = get_mapped_output(output)
    
    # convert boxes from center coordinates to corner coordinates
    out_corners = center2corner(out_yx, out_hw)
    
    # filter output (needs to be done separately for each image in batch)
    box_filtered, prob_filtered, class_filtered = tf.map_fn(
        get_filtered_output, 
        (out_corners, out_prob_obj, out_prob_class),
        dtype=(tf.float32, tf.float32, tf.int64)
    )
    
    # scale boxes from [0,1] to image space
    img_space = tf.reshape(tf.cast(tf.stack([IMG_H, IMG_W, IMG_H, IMG_W]), tf.float32), [1, 1, 4])
    box_filtered = box_filtered * img_space
    
    # perform non-max suppression
    max_objects = tf.ones(tf.shape(box_filtered)[0], dtype=tf.int32) * MAX_OBJECTS
    box_nms_indices = tf.map_fn(lambda x: tf.image.non_max_suppression(x[0], x[1], x[2], THRESHOLD_IOU_NMS), (box_filtered, prob_filtered, max_objects), dtype=tf.int32)
    box_nms = tf.map_fn(lambda x: tf.gather(x[0], x[1]), (box_filtered, box_nms_indices), dtype=tf.float32)
    prob_nms = tf.map_fn(lambda x: tf.gather(x[0], x[1]), (prob_filtered, box_nms_indices), dtype=tf.float32)
    class_nms = tf.map_fn(lambda x: tf.gather(x[0], x[1]), (class_filtered, box_nms_indices), dtype=tf.int64)
    
    return box_nms, prob_nms, class_nms

In [ ]:
# def get_mapped_output(output):
#     anchors = tf.cast(tf.reshape(ANCHORS, [1, 1, 1, ANCHORS.shape[0], 2]), dtype=tf.float32)  # try using variable for ANCHORS
    
#     # create cartesian grid on output space
#     coordinates_y = tf.range(GRID_H)
#     coordinates_x = tf.range(GRID_W)
#     x, y = tf.meshgrid(coordinates_x, coordinates_y)
#     x = tf.reshape(x, [-1])
#     y = tf.reshape(y, [-1])
#     coordinates = tf.stack([y, x], 1)
#     coordinates = tf.reshape(coordinates, [1, GRID_H, GRID_W, 1, 2])
#     coordinates = tf.cast(coordinates, tf.float32)
    
#     # change dimensions of tensors for broadcasting
#     output = tf.reshape(output, [-1, GRID_H, GRID_W, ANCHORS.shape[0], 5+NUM_OBJECTS])
#     shape = tf.cast(tf.reshape(tf.shape(output)[1:3], [1, 1, 1, 1, 2]), tf.float32)
    
#     # map output to input space (also split yx, hw, prob_obj, prob_class)
#     out_yx = tf.sigmoid(output[..., :2])  # sigmoid to keep centers in interval [0,1]
#     out_hw = tf.exp(output[..., 2:4])  # exp to keep values positive
#     out_yx = (out_yx + coordinates) / shape  # maps values to [0,1] input image space
#     out_hw = out_hw * anchors / shape
    
#     # get corresponding probabilities
#     out_prob_obj = tf.sigmoid(output[..., 4:5])
#     out_prob_class = tf.nn.softmax(output[..., 5:])
    
#     return out_yx, out_hw, out_prob_obj, out_prob_class
    

# def get_iou(out_yx, out_hw, targets):
#     # out_yx.shape = [BATCH_SIZE, GRID_H, GRID_W, NUM_ANCHORS, 2]
#     # targets.shape = [BATCH_SIZE, MAX_OBJECTS, 5]
    
#     # compute ious for each anchor in each grid in axis=4
#     out_yx = tf.expand_dims(out_yx, 4)
#     out_hw = tf.expand_dims(out_hw, 4)
#     out_min = out_yx - out_hw/2.
#     out_max = out_yx + out_hw/2.
    
#     targets = tf.reshape(targets, [tf.shape(targets)[0], 1, 1, 1, MAX_OBJECTS, 5])
#     targets_yx = targets[..., 0:2]
#     targets_hw = targets[..., 2:4]
    
#     targets_min = targets_yx - targets_hw/2.
#     targets_max = targets_yx + targets_hw/2.
    
#     intersection_min = tf.maximum(out_min, targets_min)
#     intersection_max = tf.minimum(out_max, targets_max)
#     intersection_hw = tf.maximum(intersection_max - intersection_min, 0.)
#     area_intersection = intersection_hw[..., 0] * intersection_hw[..., 1]
    
#     area_out = out_hw[...,0] * out_hw[...,1]
#     area_targets = targets[...,0] * targets[...,1]
#     area_union = area_targets + area_out - area_intersection
#     iou = area_intersection / area_union
    
#     return iou
    

# def get_confidence_loss(out_flag_object, flag_object, out_prob_obj):
#     mask_object_absent = (1 - out_flag_object) * (1 - flag_object)
#     loss_object_absent = mask_object_absent * tf.square(out_prob_obj)
    
#     loss_object_present = flag_object * tf.square(1-out_prob_obj)
#     return loss_object_absent + SCALE_LOSS_CONFIDENCE_OBJECT_PRESENT*loss_object_present
    
# def get_classification_loss(flag_object, box_adjustments, out_prob_class):
#     classes = tf.cast(box_adjustments[...,4], tf.int32)
#     classes = tf.one_hot(classes, NUM_OBJECTS)
    
#     loss_classification = flag_object * tf.square(classes - out_prob_class)
    
#     return loss_classification

# def get_regression_loss(out_box_adjustments, flag_object, box_adjustments):
#     loss_regression = flag_object * tf.square(box_adjustments[...,:4] - out_box_adjustments)
    
#     return loss_regression


    # split labels 
#     mask_object = tf.cast(tf.reshape(labels[...,5], [-1, GRID_H, GRID_W, NUM_ANCHORS]), tf.bool)
#     labels_box = labels[...,0:5]
    
#     # apply mask to differentiate object vs no_object areas
#     labels_masked = tf.boolean_mask(labels_box, mask_object)
#     predictions_masked = tf.boolean_mask(predictions, mask_object)
#     predictions_masked_negative = tf.boolean_mask(predictions, tf.logical_not(mask_object))
    
#     # apply corresponding transformations on predictions
#     predictions_masked_yx = tf.sigmoid(predictions_masked[...,0:2])
#     predictions_masked_hw = tf.exp(predictions_masked[...,2:4])
#     predictions_masked_obj = tf.sigmoid(predictions_masked[...,4])
#     predictions_masked_no_obj = tf.sigmoid(predictions_masked_negative[...,4])
#     predictions_masked_class = tf.nn.softmax(predictions_masked[...,5:])
    
#     # split labels_box
#     labels_masked_yx = labels_masked[...,0:2]
#     labels_masked_hw = labels_masked[...,2:4]
#     labels_masked_class = labels_masked[...,4]
#     labels_masked_one_hot = tf.reshape(tf.one_hot(tf.cast(labels_masked_class, tf.int32), NUM_OBJECTS), [-1, NUM_OBJECTS])
    
#     # compute loss
#     loss_object = tf.reduce_sum(tf.square(predictions_masked_obj - 1))
#     loss_no_object = tf.reduce_sum(tf.square(predictions_masked_no_obj))
#     loss_yx = tf.reduce_sum(tf.square(predictions_masked_yx - labels_masked_yx))
#     loss_hw = tf.reduce_sum(tf.square(tf.sqrt(predictions_masked_hw) - tf.sqrt(labels_masked_hw)))
#     loss_class = tf.reduce_sum(tf.square(predictions_masked_class - labels_masked_one_hot))
    
#     loss = tf.cast(1/tf.shape(predictions)[0], tf.float32) * (COOEFFICIENT_OBJ*loss_object + COOEFFICIENT_NO_OBJ*loss_no_object + COOEFFICIENT_REG*(loss_yx + loss_hw) + loss_class)
############################
#     # map output from grid space to [0,1] space
#     out_yx, out_hw, out_prob_obj, out_prob_class = get_mapped_output(output)
    
#     # get iou for all boxes wrt each anchor in each grid
#     iou = get_iou(out_yx, out_hw, targets)
    
#     # keep best bbox for each anchor for each grid as per iou
#     iou_best = tf.reduce_max(iou, axis=4, keepdims=True)
    
#     # threshold over ious
#     out_flag_object = tf.cast(iou_best > THRESHOLD_IOU, tf.float32)
    
#     # object presence loss (confidence)
#     loss_confidence = get_confidence_loss(out_flag_object, flag_object, out_prob_obj)
    
#     # classification loss
#     loss_classification = get_classification_loss(flag_object, box_adjustments, out_prob_class)
    
#     # regression loss
#     out_box_adjustments = tf.concat([tf.sigmoid(output[...,0:2]), tf.exp(output[...,2:4])], axis=-1)
#     loss_regression = get_regression_loss(out_box_adjustments, flag_object, box_adjustments)
    
#     # accumulate loss
#     loss_confidence_sum = tf.reduce_sum(loss_confidence)
#     loss_classification_sum = tf.reduce_sum(loss_classification)
#     loss_regression_sum = tf.reduce_sum(loss_regression)
    
#     loss = tf.cast(1/tf.shape(targets)[0], tf.float32) * 0.5 * (loss_confidence_sum + loss_classification_sum + loss_regression_sum)
    
    
#     return loss

In [ ]:
# tf.reset_default_graph()
# output = tf.random_normal([BATCH_SIZE, GRID_H, GRID_W, NUM_ANCHORS, 5+20])
# b, p, c = get_predictions(output)

# with tf.Session() as sess:
#     b, p, c = sess.run([b, p, c])

In [ ]:
# tf.reset_default_graph()
# f, l = input_fn_train(DATA_TRAIN)

# with tf.Session() as sess:
#     f, l = sess.run([f,l])

# idx_in_batch = 0
# l = l['targets'][idx_in_batch]

# plt.imshow(f['img'][0,:,:,:])
# for idx_object in range(l.shape[0]):
#     plt.scatter((l[idx_object,1]-l[idx_object,3]/2)*IMG_W, (l[idx_object,0]-l[idx_object,2]/2)*IMG_H, color='w')
#     plt.scatter((l[idx_object,1]-l[idx_object,3]/2)*IMG_W, (l[idx_object,0]+l[idx_object,2]/2)*IMG_H, color='w')
#     plt.scatter((l[idx_object,1]+l[idx_object,3]/2)*IMG_W, (l[idx_object,0]-l[idx_object,2]/2)*IMG_H, color='w')
#     plt.scatter((l[idx_object,1]+l[idx_object,3]/2)*IMG_W, (l[idx_object,0]+l[idx_object,2]/2)*IMG_H, color='w')
# plt.show()